In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import math
from collections import Counter

In [3]:
class Node():
  def __init__(self,feature=None, left=None,    right=None,    value=None, threshold=None):
    self.feature=feature
    self.left=left
    self.right=right
    self.value=value
    self.threshold=threshold
  def __str__(self):
    return "feature:"+str(self.feature) + " threshold:" + str(self.threshold) + " left:" + str(self.left) +" right:" + str(self.right)


In [6]:
class Decision():

  #for getting value of 1/0 for wine quality
  def get_max_count(self,dataset):
    count=0
    # print("max count is +++\n",dataset)

    for i in dataset:
      # print("i is", i)
      if i>=7:
        count+=1
    return  1 if count>(len(dataset)-count) else 0

  def entropy(self,nparray):
    if len(nparray)==0:
      return 0

    total_length=len(nparray)
    count=0

    #counting positives and negatives in the dataset
    for i in nparray:
      if i>=7:
        count+=1
    if count==0  or total_length-count==0:
      return 0
    # print("count",count)
    return -((count/total_length)*math.log2(count/total_length)+((total_length-count)/total_length)*math.log2((total_length-count)/total_length))

  #take a dataset for a particular threshold divide it between two data set
  def split(self,dataset,feature,threshold):
    # print("{{{}}}",dataset)
    leftsplit=[]
    rightsplit=[]

    for row in dataset:
      # print(row,type(row))
      if row[feature]<=threshold:
        leftsplit.append(row)
      else:
        rightsplit.append(row)
    return np.array(leftsplit),np.array(rightsplit)

  def information_gain(self,parent, left, right):
    """Calculate the information gain for a split."""
    # Calculate entropy of the parent
    parent_entropy = self.entropy(parent)

    # Calculate the weight of the left and right splits
    left_weight = len(left) / len(parent)
    right_weight = len(right) / len(parent)

    # Calculate the entropy of the left and right children
    left_entropy = self.entropy(left)
    right_entropy = self.entropy(right)

    # Calculate weighted entropy of the children
    weighted_entropy = left_weight * left_entropy + right_weight * right_entropy

    # Information gain
    info_gain = parent_entropy - weighted_entropy
    return info_gain

  def get_best_split(self,dataset):
    no_feature=dataset.shape[1]
    # print("no of features",no_feature)

    maxgain=-1
    max_threshold=None
    max_feature=None

    # print("No of features",no_feature)
    pleft=None
    pright=None

    for col in range(no_feature-1):
      # print("OOOOO",col,dataset[:,col])
      to_test_thresholds=np.unique(dataset[:,col])
      # print("threshold iteration",len(to_test_thresholds))
      for i in to_test_thresholds:
        left,right=self.split(dataset,col,i)
        # print("left",len(left))
        # print("right",len(right))

        if len(left) and len(right):
        # get y values of the parent and left, right nodes
          y, left_y, right_y = dataset[:, -1], left[:, -1], right[:, -1]
          inf_gain=self.information_gain(y,left_y,right_y)
          # print("information gain",inf_gain)

          #entropu and information gain is fucking up please check!!!!!!!!
          if maxgain<inf_gain:
            max_threshold=i
            maxgain=inf_gain
            max_feature=col
            pleft=left
            pright=right

        # maxgain=max(maxgain,information_gain(dataset,left,right))
      # print(col, len(np.unique(df[col])))
    # print("maxgain",maxgain)
    # print("max_threshold",max_threshold)
    # print("max_feature",max_feature)
    return pleft,pright,max_threshold,max_feature,maxgain
    pass

  def build_tree(self,dataset,current_depth=0):
    if current_depth>4 or len(dataset)<2:
      # print("LLLLLLLL")
      return Node(value=self.get_max_count(dataset[:,-1]))
    else:
      left,right,threshold,feature,gain=self.get_best_split(dataset)
      if gain==0:
        return Node(value=self.get_max_count(dataset[:,-1]))

      left=self.build_tree(left,current_depth+1)
      right=self.build_tree(right,current_depth+1)
      return Node(feature=feature,threshold=threshold,left=left,right=right)
      pass

  def predict(self,node,row):
    # print("row",row)
    if node.value!=None:
      return node.value

    # if node.feature is not None and row[node.feature]<=node.threshold:
    if row[node.feature]<=node.threshold:
      return self.predict(node.left,row)
    else:
      return self.predict(node.right,row)


  def test(self,dataset,root):
    out=[]
    for i in dataset:
      out.append(self.predict(root,i))

    return np.array(out)
    pass


In [8]:
path="/content/drive/MyDrive/winequality-red.csv"
df=pd.read_csv(path,sep=';')
# print(df.shape[1])
# df=df[["fixed acidity","volatile acidity","quality"]]
# print("df",df[1:20])

#                             training set 90%
df=df[:1350]
# print("lastcoumn",df.iloc[:,-1])
# for_entropy=np.array(df.iloc[:,-1])

mytree=Decision()
# print("entropy",mytree.entropy(for_entropy))
# mytree.entropy(df[1:100])
# print(mytree.get_best_split(np.array(df)))

root=mytree.build_tree(np.array(df))

print(df)
df=pd.read_csv(path,sep=';')
# df=df[["fixed acidity","volatile acidity","quality"]]

#                       testing set 10%
df=df[1351:]
# print("PPPPPPPPPPPPPPPP",df)
X_test=np.array(df.iloc[:, :-1])
Y_test=np.array(df.iloc[:,-1])
# print("xtest",X_test,"ytest",type(Y_test))

y_pred=mytree.test(X_test,root)
# print("predicted ans",y_pred)
newtest=[]
for i in Y_test:
  # print(i)
  if i>=7:
    newtest.append(1)
    # check[i]=check.get(i,0)+1
  else:
    newtest.append(0)
    # check[i]=check.get(i,0)+1
# print(newtest)
ytest=np.array(newtest)
# print("req",ytest)
# ytest=np.array([1 if i>7 else 0 for i in y_test])
# print("req",ytest)


#   # y_pred=test(X_test,root)

accuracy = np.mean(ytest == y_pred)
# scores.append(accuracy)
print(f"Accuracy: {accuracy * 100:.2f}%")

# l,r=mytree.split(np.array(df),0,7.3)
# print("left",l)
# print("right",r)


      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1345            8.2             0.440         0.24             2.3      0.063   
1346            6.1             0.590         0.01             2.1      0.056   
1347            7.2             0.655         0.03             1.8      0.078   
1348            7.2             0.655         0.03             1.8      0.078   
1349            6.9             0.570         0.00             2.8      0.081   

      free sulfur dioxide  